# Prescriber View - Sales Activity

In [1]:
import polars as pl
import pandas as pd
import gc
import numpy as np
from datetime import datetime, timedelta,date
import json

In [2]:
# load variables from JSON
with open('vars_wk.json', 'r') as json_file:
    js = json.load(json_file)

bucket = js['bucket']
dflib = f's3://{bucket}/BIT/dataframes/'

In [3]:
# Utility Functions -
def load(df, lib=dflib):
    globals()[df] = pl.read_parquet(f'{lib}{df}.parquet')

In [4]:
# Imporing Dependencies
# loading QTD Feed Data from weekly view - >
weekly_qtd_psa = pl.read_parquet(f'{dflib}PSA_qtd.parquet')

In [5]:
#period_loop_lookup_dict (PN 5 has no data in the feed so skipping)
pld = {1:'1-MONTH',2:'3-MONTHS',3:'6-MONTHS',4:'12-MONTHS',6:'PQTD',7:'YTD'}
# export location - >
OUT = 's3://vortex-staging-a65ced90/BIT/output/Prescriber/Monthly/'

In [6]:
for period_num,PN in zip([1,3,6,12,'pqtr','ytd'],[1,2,3,4,6,7]):
    temp1 = (
        weekly_qtd_psa
        .with_columns(
            pl.lit('MONTHLY').alias('ReportType'),
            pl.lit(f'{pld[PN]}').alias('Period')
        )
    )
    temp1 = temp1.to_pandas()
    # Select columns of type 'object' (string)
    string_columns = temp1.select_dtypes(include=['object']).columns.tolist()
    temp1[string_columns] = temp1[string_columns].fillna('\\N')
    temp1 = temp1.replace('NaN', '\\N')
    
    temp1 = temp1.replace([np.nan, np.inf, -np.inf], '\\N')
    temp1.to_csv(f'{OUT}Monthly_Prescriber_SalesActivity_P{PN}_Feed.txt', sep='|',lineterminator='\r\n',index=False)
    print(f'Exported Feed {PN}')

Exported Feed 1


Exported Feed 2


Exported Feed 3


Exported Feed 4


Exported Feed 6


Exported Feed 7
